In [62]:
import torch
import torch.nn as nn

Embedding with batch size $1$ and token sequences of different length.

In [63]:
#supponiamo di avere tre pazienti con EHRs di lunghezze diverse 
#vogliamo applicare CBOW considerando il contesto K (user-defined) separatamente per ogni "sentence",
#cioe' separatamente per ogni paziente
#Creiamo one list with different arrays storing the tokens of the medical terms
# one array for each patient
P1 = "A wiki is run using wiki software, otherwise known as a wiki engine.\
         A wiki engine is a type of content management system, but it differs\
         from most other such systems, including blog software, in that the\
         content is created without any defined owner or leader, and wikis have\
         little implicit structure, allowing structure to emerge according to the\
         needs of the users."

P2 = "The online encyclopedia project Wikipedia is by far the most popular wiki-based\
         website, and is one of the most widely viewed sites of any kind in the world,\
         having been ranked in the top ten since 2007."

P3 = "Wikipedia is not a single wiki but rather a collection of hundreds of wikis,\
         one for each language. There are tens of thousands of other wikis in use, both\
         public and private, including wikis functioning as knowledge management resources,\
         notetaking tools, community websites and intranets. The English-language Wikipedia\
         has the largest collection of articles; as of September 2016, it had over five\
         million articles."

In [64]:
##ARRAY with tokens from patients
H = [P1.split(), P2.split(), P3.split()]

In [65]:
##Fun1: creiamo il nostro dictionary (associamo univocamente un token a un intero)
##Input: array of sentences
def create_dict(array_sentences): 
    word_to_ix = {}

    for _, j in enumerate(array_sentences):
        for word in j:
            if word not in word_to_ix:
                word_to_ix[word] = len(word_to_ix)
    return word_to_ix

In [66]:
word_to_ix = create_dict(H)

In [67]:
data = [[word_to_ix[mt] for mt in a] for _, a in enumerate(H)]

In [68]:
embedding = nn.Embedding(len(word_to_ix), 10)

In [69]:
##TRAINING
EMBEDDING_DIM = 10
vocab_size = len(word_to_ix)
embedding = nn.Embedding(len(word_to_ix), 10)

loss_function = nn.NLLLoss()
optimizer = torch.optim.SGD(embedding.parameters(), lr=0.001)

In [70]:
for epoc in range(50):
    total_loss = 0
    for sequence in data:
        embedding.zero_grad()
        sequence = torch.tensor(sequence)
        log_probs = embedding(sequence)
        loss = loss_function(log_probs, torch.LongTensor([word_to_ix[target]]))
        loss.backward()
        optimizer.step()
        
        total_loss += loss.data

NameError: name 'target' is not defined

In [36]:
def get_word_embedding(word):
        word = torch.LongTensor([word_to_ix[word]])
        return embedding(word).view(1, -1)

In [37]:
def pat_embedding(array_sentences, emb_dim, model):
    mat_all = []

    for sentence in array_sentences:
        emb = torch.empty(len(sentence), emb_dim)
        for med_term in sentence:
            emb = torch.cat((emb, get_word_embedding(med_term).detach()),0)
        mat_all += [emb]

    return mat_all

In [71]:
EMBEDDING_DIM = 10
mat_all = pat_embedding(H, EMBEDDING_DIM, embedding)

In [50]:
print(mat_all[0].size())
len(H[0])

torch.Size([126, 10])


63

In [72]:
embedding.weight

Parameter containing:
tensor([[-0.4013, -0.6560,  0.0649,  ...,  0.2077,  2.2426, -0.7861],
        [ 0.5487,  0.1065,  1.4480,  ...,  0.5269,  1.3526, -0.0994],
        [ 0.6229,  0.8686,  0.5328,  ..., -0.0897,  0.2653, -1.1034],
        ...,
        [-0.6540, -0.4573,  2.6081,  ...,  0.9091, -0.7375, -2.2961],
        [-1.8820, -1.5328, -0.0443,  ...,  0.7891, -0.5312, -1.5133],
        [ 1.7572, -0.9223,  1.4437,  ...,  0.0981,  1.2850, -0.6266]])

In [22]:
for i, _ in enumerate(data):
    embedding(torch.LongTensor(data[i]))

Parameter containing:
tensor([[-0.7658, -1.1076, -1.0605,  ..., -0.1243, -0.3123,  0.9120],
        [ 0.1745, -0.9270,  0.0739,  ..., -0.9071,  2.3402, -0.2193],
        [ 0.4127,  1.6022,  1.6502,  ..., -0.7562,  0.0355, -0.7286],
        ...,
        [ 1.5872, -0.9021, -0.8099,  ...,  0.1343,  0.4993, -0.6932],
        [ 0.3248,  0.2295, -0.8391,  ...,  0.6187,  2.4658, -0.8536],
        [ 0.3259, -1.0615,  0.3930,  ...,  0.2402, -0.3682,  0.2493]])